<a href="https://colab.research.google.com/github/paruliansaragi/DL-Notebooks/blob/master/OxfordPractical2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import numpy as np
import os
from random import shuffle
import re
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
import collections
import time
import pandas as pd
import urllib.request
import zipfile
import lxml.etree

In [0]:
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [0]:
# extract both the texts and the labels from the xml file
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
texts = doc.xpath('//content/text()')
labels = doc.xpath('//head/keywords/text()')
del doc

In [0]:
# preprocess the labels: search for occurences of the keywords "technology", "entertainment" or "design" and build labels
label_lookup = ['ooo', 'Too', 'oEo', 'ooD', 'TEo', 'ToD', 'oED', 'TED']
for i in range(len(labels)):
    ted_labels = ['o', 'o', 'o']
    keyword_list = labels[i].split(', ')
    if 'technology' in keyword_list:
        ted_labels[0] = 'T'
    if 'entertainment' in keyword_list:
        ted_labels[1] = 'E'
    if 'design' in keyword_list:
        ted_labels[2] = 'D'
    labels[i] = ''.join(ted_labels)
    labels[i] = label_lookup.index(labels[i])
len(labels)
labels[:20]

[0, 0, 0, 3, 5, 0, 0, 0, 0, 5, 0, 3, 2, 5, 0, 0, 3, 0, 5, 0]

In [0]:
#Map index to Label name
#label_text = []
#for i in labels:
#  label_text.append(label_lookup[i])

In [0]:
# preprocess the texts: lowercase, remove text in parentheses, remove punctuation, 
#tokenize into words (split on whitespace)

#removing text in parentheses
input_texts = [re.sub(r'\([^)]*\)', '', input_text) for input_text in texts]
#lowercase
input_texts = [input_text.lower() for input_text in input_texts]
#remove punctuation
input_texts = [re.sub(r'[^a-z0-9]+', ' ', input_text) for input_text in input_texts]
#tokenize into words
input_texts = [input_text.split() for input_text in input_texts]
len(input_texts)

2085

In [0]:
vocab_list = list(set([word for input_text in input_texts for word in input_text]))
word_to_index = {}
index_to_word = {}
for i, word in enumerate(vocab_list):
    word_to_index[word] = i
    index_to_word[i] = word
input_indices_list = []
for input_text in input_texts:
    input_indices_list.append([word_to_index[word] for word in input_text])
len(vocab_list)

54778

https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [0]:
# combining the tokens and labels for each input, then shuffle them and split into train/test/cv
inputs_combined = list(zip(texts, labels))
df = pd.DataFrame(inputs_combined, columns=['Texts', 'Labels'])

In [0]:
df.head()

,Texts,Labels
0,Here are two reasons companies fail: they only...,0
1,So there are lands few and far between on Eart...,0
2,"OK, so, confession: I've always been weirdly o...",0
3,It has been 128 years since the last country i...,3
4,"Thousands of years from now, we'll look back a...",5


In [0]:
from sklearn.model_selection import train_test_split

trn, test = train_test_split(df, test_size=0.1)

https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/08.CNN-for-Text-Classification.ipynb

In [0]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [0]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [0]:
df.head()

,Texts,Labels
0,Here are two reasons companies fail: they only...,0
1,So there are lands few and far between on Eart...,0
2,"OK, so, confession: I've always been weirdly o...",0
3,It has been 128 years since the last country i...,3
4,"Thousands of years from now, we'll look back a...",5


In [0]:
list_sentences_train = trn["Texts"].fillna("_na_").values
y = trn["Labels"].values
list_sentences_test = test["Texts"].fillna("_na_").values

In [0]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
len(df.Labels.unique())

8

In [0]:
y_train = keras.utils.to_categorical(trn.Labels, 8)


In [0]:
y_train[0], y_train[1]

(array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

In [0]:
X_t.shape, y.shape

((1876, 100), (1876,))

In [0]:
path = './'
EMBEDDING_FILE=f'{path}glove.6B.50d.txt'

In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-01-21 21:18:49--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-01-21 21:18:49--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1      100%[===================>] 822.24M  11.0MB/s    in 73s     

2019-01-21 21:20:02 (11.3 MB/s) - ‘glove.6B.zip.1’ saved [862182613/862182613]



In [0]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [0]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [0]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(8, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#Future idea: try drop connections as a better method of regularization than dropout.

In [0]:
model.fit(X_t, y_train, batch_size=32, epochs=5, validation_split=0.1);

Train on 1688 samples, validate on 188 samples
Epoch 1/5
1688/1688 [==============================] - 32s 19ms/step - loss: 0.3700 - acc: 0.8621 - val_loss: 0.2967 - val_acc: 0.8763
Epoch 2/5
1688/1688 [==============================] - 28s 17ms/step - loss: 0.2859 - acc: 0.8842 - val_loss: 0.2825 - val_acc: 0.8883
Epoch 3/5
1688/1688 [==============================] - 28s 17ms/step - loss: 0.2804 - acc: 0.8871 - val_loss: 0.2810 - val_acc: 0.8816
Epoch 4/5
1688/1688 [==============================] - 28s 16ms/step - loss: 0.2767 - acc: 0.8921 - val_loss: 0.2783 - val_acc: 0.8783
Epoch 5/5
1688/1688 [==============================] - 28s 17ms/step - loss: 0.2650 - acc: 0.8981 - val_loss: 0.2805 - val_acc: 0.8803


In [0]:
y_test = model.predict([X_te], batch_size=1024, verbose=1); y_test

209/209 [==============================] - 1s 5ms/step


array([[0.61623055, 0.17294072, 0.06487866, ..., 0.04402405, 0.00857992,
        0.01092159],
       [0.7335242 , 0.16294411, 0.06699197, ..., 0.03705277, 0.01086555,
        0.01639839],
       [0.3683236 , 0.29769585, 0.19367303, ..., 0.08433745, 0.01835334,
        0.0235731 ],
       ...,
       [0.75984985, 0.14874512, 0.07800739, ..., 0.03821587, 0.01300069,
        0.01815613],
       [0.37052464, 0.27739808, 0.08840923, ..., 0.07980302, 0.01008764,
        0.01740964],
       [0.78692466, 0.13244998, 0.05725488, ..., 0.02869911, 0.00957986,
        0.01459876]], dtype=float32)

In [0]:
np.argmax(y_test[1])

0

# Predict Text from label

In [0]:
X_t.shape, y_train.shape

((1876, 100), (1876, 8))

In [0]:
model.fit(y_train, X_t, batch_size=32, epochs=5, validation_split=0.1);

Train on 1688 samples, validate on 188 samples
Epoch 1/5
1688/1688 [==============================] - 5s 3ms/step - loss: -1685.5170 - acc: 0.0353 - val_loss: -7305.7932 - val_acc: 0.0437
Epoch 2/5
1688/1688 [==============================] - 3s 2ms/step - loss: -12989.4064 - acc: 0.0416 - val_loss: -13746.7209 - val_acc: 0.0437
Epoch 3/5
1688/1688 [==============================] - 3s 2ms/step - loss: -14132.0390 - acc: 0.0416 - val_loss: -13746.7209 - val_acc: 0.0437
Epoch 4/5
1688/1688 [==============================] - 3s 2ms/step - loss: -14132.0428 - acc: 0.0416 - val_loss: -13746.7209 - val_acc: 0.0437
Epoch 5/5
1688/1688 [==============================] - 3s 2ms/step - loss: -14131.9199 - acc: 0.0416 - val_loss: -13746.7209 - val_acc: 0.0437


In [0]:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
rev_label = doc.xpath('//head/keywords/text()')
del doc

In [0]:
inputs_combined = list(zip(texts, rev_label))
df_rev = pd.DataFrame(inputs_combined, columns=['Texts', 'Labels'])

In [0]:
df_rev.head()

,Texts,Labels
0,Here are two reasons companies fail: they only...,"talks, business, creativity, curiosity, goal-s..."
1,So there are lands few and far between on Eart...,"talks, Planets, TEDx, bacteria, biology, engin..."
2,"OK, so, confession: I've always been weirdly o...","talks, Debate, Guns, activism, big problems, c..."
3,It has been 128 years since the last country i...,"talks, Brazil, Slavery, art, beauty, community..."
4,"Thousands of years from now, we'll look back a...","talks, NASA, communication, computers, creativ..."


In [0]:
trn_rev, test_rev = train_test_split(df_rev, test_size=0.1)

In [0]:
list_of_labels_trn = trn_rev["Labels"].fillna("_na_").values
y_rev = trn_rev["Texts"]
list_of_labels_test = test_rev["Labels"].fillna("_na_").values

In [0]:
y_rev[:5]

1307    I grew up in a very small village in Canada, a...
1070    I was offered a position as associate professo...
111     (Guitar music)\nI was just thinking that I hav...
659     When I was a young boy, I used to gaze through...
1822    To be new at TED -- it's like being the last h...
Name: Texts, dtype: object

In [0]:
list_of_labels_trn[:5]

array(['talks, culture, design, education, green',
       'talks, design, medical research, science, visualizations',
       'talks, guitar, live music, music, performance, piano',
       'talks, TEDx, animals, anthropology, deextinction, evolution',
       'talks, TED Conference, architecture, art, business, design, humor'],
      dtype=object)

In [0]:
rev_tokenizer = Tokenizer(num_words=max_features)
rev_tokenizer.fit_on_texts(list(y_rev))
rev_list_tokenized_train = tokenizer.texts_to_sequences(list_of_labels_trn)
rev_list_tokenized_test = tokenizer.texts_to_sequences(list_of_labels_test)
rev_X_t = pad_sequences(rev_list_tokenized_train, maxlen=8)
rev_X_te = pad_sequences(rev_list_tokenized_test, maxlen=maxlen)

In [0]:
rev_X_t[:5]

In [0]:
rev_tldr = tokenizer.texts_to_sequences(y_rev)
rev_tldr_X_t = pad_sequences(rev_tldr, maxlen=maxlen)

In [0]:
rev_tldr_X_t[:5]

In [0]:
rev_word_index = rev_tokenizer.word_index
rev_nb_words = min(max_features, len(rev_word_index))
rev_embedding_matrix = np.random.normal(emb_mean, emb_std, (rev_nb_words, embed_size))
for word, i in rev_word_index.items():
    if i >= max_features: continue
    rev_embedding_vector = embeddings_index.get(word)
    if rev_embedding_vector is not None: rev_embedding_matrix[i] = rev_embedding_vector

In [0]:
rev_tldr_X_t.shape, rev_X_t.shape

((1876, 100), (1876, 8))

In [0]:
#we want to predict text based on label

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(8, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#categorical cross entropy might work better
#Future idea: try drop connections as a better method of regularization than dropout.

In [0]:
model.fit(rev_X_t, rev_tldr_X_t, batch_size=32, epochs=5, validation_split=0.1);